In [1]:
# -*- coding: utf-8 -*-
import os
import pathlib
from anytree import Node, RenderTree
from anytree.resolver import Resolver
import re

![](img_9561_robert_confused.jpg)

Import data

In [52]:
pagesdir = os.path.join(os.getcwd(),'pages')
media_ondisk = set()
with open('medialist.txt', 'r') as f:
    lines = f.read().split("\n")
    for line in lines:
        if "." in line:
            media_ondisk.add(line[6:].lower())

In [40]:
def is_template(filename):
    return (file.startswith('fr_') \
            or file.startswith('f_') \
            or file.startswith('_') \
            or file.startswith('n_'))

Any directories (namespaces) that do not contain child-namespaces or pages?

In [93]:
count = 0
for root, dirs, files in os.walk(pagesdir):  
    count += len(files)
print("{} pages".format(count))

count_empty = 0
for root, dirs, files in os.walk(pagesdir):  
    if len(files) == 0:
        count_empty += 1
        print(root)
print("{} empty namespaces".format(count_empty))

28571 pages
/home/jens/git/wikiminer/pages/namensraum
/home/jens/git/wikiminer/pages/namensraum/infovault
/home/jens/git/wikiminer/pages/namensraum/infovault/bilder
/home/jens/git/wikiminer/pages/namensraum/infovault/bilder/2017
/home/jens/git/wikiminer/pages/fahrwerk/kit15/fahrwerk
/home/jens/git/wikiminer/pages/fahrwerk/kit15/fahrwerk/kit15
/home/jens/git/wikiminer/pages/fahrwerk/kit16/konzept
/home/jens/git/wikiminer/pages/fahrwerk/bremssaettel
/home/jens/git/wikiminer/pages/fahrwerk/antriebswelle/kit17
/home/jens/git/wikiminer/pages/fahrwerk/radtraeger/kit18/konzept_e
/home/jens/git/wikiminer/pages/fahrwerk/validierung
/home/jens/git/wikiminer/pages/fahrwerk/validierung/kit17
/home/jens/git/wikiminer/pages/fahrwerk/fahrdynamiksimulation/kit13/fahrwerk
/home/jens/git/wikiminer/pages/fahrwerk/fahrdynamiksimulation/kit13/fahrwerk/fahrdynamiksimulation
/home/jens/git/wikiminer/pages/fahrwerk/fahrdynamiksimulation/kit13/fahrwerk/fahrdynamiksimulation/kit13
/home/jens/git/wikiminer/pages

Regexes for links and embedded media

In [48]:
## REGEX EXPRESSIONS
# matches any links, except signatures
re_link=re.compile(r'\[\[(?!.*\@ka-raceing\.de.*)(.*?)\]\]')
# matches any embedded media, complete link with sizes and title
#re_embeddedmedia=re.compile(r'\{\{(.*?)\}\}')
# matches any embedded media, only filename
re_embeddedmedia=re.compile(r'\{\{(.*?)[\||\?|}].*?\}\}')

Classes

In [83]:
class Namespace(Node): 
    """
    A namespace has a name and contains pages
    """
    def __init__(self, name, parent=None, pages=[]):
        super().__init__(name, parent)
        self.pages = pages      
               
    def __repr__(self):
        return '<ns> {}'.format(self.name)
        
class Wikipage:
    """
    A wikipage has a name and source code. 
    It can also generate a list of links and media from the source.
    """
    
    def __init__(self, name, src, populate_immediately=False):
        self.name = name
        self.src = src
        if populate_immediately:
            self.links = set(self.getlinks())
            self.media = set(self.getmedia())
        else:
            self.links = set()
            self.media = set()
        
    def __repr__(self):
        return('{} [{} links, {} media]'.format(self.name,
                                                         len(self.links),
                                                         len(self.media)))
    def getlinks(self):
        srclinks = re_link.findall(self.src)
        links = []
        
        for match in srclinks:
            try:
                link, src = match.split('|')
            except ValueError:
                link = match
                src = ''
            links.append((link,src))
        return links
        
    def getmedia(self):
        return re_embeddedmedia.findall(self.src)

Build tree

In [84]:
r = Resolver('name')    

rootns = Namespace(name = 'pages', parent=None)
for root, dirs, files in os.walk(pagesdir): 
    relpath = os.path.relpath(root, pagesdir)
    
    if relpath == '.':
        basens = rootns
    else:
        basens = r.get(rootns, relpath)
        
    for directory in dirs:
        ns = Namespace(name=directory, parent=basens)
        
    basens.pages = []
    for file in files:
        if not is_template(file):
            with open(os.path.join(root, file), encoding=('utf-8')) as f:
                pagesrc = f.read()
                pagename = os.path.splitext(file)[0]
                thispage = Wikipage(name=pagename, src=pagesrc, 
                                   populate_immediately=True)
                basens.pages.append(thispage)

In [96]:
PRINT = False
if PRINT:
    for pre, fill, node in RenderTree(rootns):
        print("{}{}".format(pre, node.name.upper()))
        if len(node.pages) == 1:
            print("{} * {}".format(pre, node.pages[0]))
        if len(node.pages) > 1:
            for page in node.pages[1:]:
                print("{} * {}".format(fill, page))

Create media lists: internally linked media, externally linked media, and pages which use the gallery plugin

Dokuwiki does not care about capitalization, and it also replaces any umlauts or ß with the anglisized versions. So we will do that as well in the media list.
Furthermore, __ in any links is replaced with _ , as in infovault/bilder/2014/sponsoren/etas/fsg/fotowettbewerb/etas__0001.jpg

In [97]:
from anytree import PreOrderIter

allmedia = set()
externalmedia = set()
pages_with_galleries = set()

for namespace in PreOrderIter(rootns):
    for page in namespace.pages:
        for media in page.media:
            media = media.strip()
            if "gallery>" in media:
                pages_with_galleries.add((namespace.path, page))
            elif 'http' in media or 'www.' in media:
                if media.startswith('https://wiki.ka-raceing.de/_media/'):
                    allmedia.add(media[34:].replace(":","/"))
                else:
                    externalmedia.add(media)
            else:
                media = media.lower()
                media = media.replace("ä", "ae")
                media = media.replace("ö", "oe")
                media = media.replace("ü", "ue")
                media = media.replace("ß", "ss")
                media.replace("__","_")
                if media[0] == ":":
                    allmedia.add(media[1:].replace(":","/"))
                else:
                    allmedia.add(media.replace(":","/"))

Pages which use the gallery plugin

In [98]:
pages_with_galleries = ["/".join([n.name for n in p[0][1:]]) + "/" + p[1].name for p in pages_with_galleries]
for p in sorted(pages_with_galleries):
    print(p)

driverless/protokolle/2017-01-15-battery
elektroauto/batterie/kit15/fertigung/ableiter-verschweissen-imvt
elektroauto/batterie/kit15/fertigung/abrissschutz
elektroauto/batterie/kit15/fertigung/bilder
elektroauto/batterie/kit15/fertigung/containerform
elektroauto/batterie/kit15/fertigung/deckelform
elektroauto/batterie/kit15/fertigung/elektronik
elektroauto/batterie/kit15/fertigung/luftauslass
elektroauto/batterie/kit15/fertigung/stacks
elektroauto/batterie/kit15/fertigung/vermessung_stromschienen
elektroauto/batterie/kit15/fertigung/zwischenwaende
elektroauto/batterie/kit15/simulation
elektroauto/batterie/kit15/simulation/iterationen
elektroauto/batterie/kit15/simulation/rbevspressure
elektroauto/batterie/kit15/testen
elektroauto/batterie/kit15/zelltest_150202
elektroauto/batterie/kit16/fertigung/containerform
elektroauto/batterie/kit16/fertigung/stromschienen
elektroauto/drehgeber_pruefstand/kit15/design
elektroauto/drehgeber_pruefstand/kit15/inbetriebnahme
elektroauto/hvmodul/kit16/f

Media that is linked, but was not found on disk

In [99]:
media_notondisk = allmedia - media_ondisk 
print(len(media_notondisk))
for l in sorted(media_notondisk):
    #if "eigen__steuer" in l:
        print(l)

1268

 aerodynamik/bodywork_c/kit15/design/seitenkasten2.1.png
*bilder*/fahrwerk/kit15/dsc_8280.jpg
*bilder*/fahrwerk/kit15/dsc_8281.jpg
/2012_06_29_racecamp.pdf
/applikation_blow_by.pdf
/artikel_oelverbrauch_am_abgasturbolader_t_m.pdf
/blow_by_messungen.pdf
/blowbyfs133.jpg
/es521223_03_general_oil_lubrication_std.pdf
/infovault/bilder/mg_0344.jpg
/mvv_praesentation_20130930.pptx
/training_30000.png
/turbo_prueftabelle.pdf
1000w-end-900cool.png
1000w-end-waterdiff.png
2014/events/fss/samstag/impressionen/img_5365.jpg
337 volt
500w-end-900cool.png
500w-end-nocool.png
[[infovault/bilder/teamfoto2015/img_3510.jpg
\rho {\bold \mathrm v
_tl_bild
aerodynamik/bodywork_e/kit15/bilder_fertigung/imag2110.jpg
aerodynamik/bodywork_e/kit15/bilder_fertigung/imag2112.jpg
aerodynamik/bremsenkuehlung/kit16/design/ohne_luftfuehrung_mit_la_fter.png
aerodynamik/cfd/kit15/simulationsverwaltung/ergebnisse_kit15e_07e_20150727 2.xlsm
aerodynamik/drs/kit18/design_c/gehaeuse_v2_links.png
aerodynamik/frontflueg

Media that is on disk, but not linked

In [100]:
media_nolinked = media_ondisk - allmedia

for l in sorted(media_nolinked):
    found = False
    for p in pages_with_galleries:
        if l[20:] in p:
            found = True
    if found: 
        media_nolinked.remove(l)
    else:
        print(l)
print(len(media_nolinked))

03_11_2010_monocoque_bauteilpraesentation_gesamt.ppt
1.4301_2mm_1x_schweisslehre_v1.dxf
1.4301_2mm_1x_schweisslehre_v1.pdf
12evo/drivetrain/kit16/design/b.png
12evo/drivetrain/kit16/design/bu.png
12evo/drivetrain/kit16/design/cl.png
12evo/drivetrain/kit16/design/cp.png
12evo/drivetrain/kit16/design/cr.png
12evo/drivetrain/kit16/design/kraefte_berechnung_diffhalter_kit10.xlsx
12evo/drivetrain/kit16/design/ktm_450_sxf_sprocket.jpg
12evo/drivetrain/kit16/design/optdiff_b.png
12evo/drivetrain/kit16/design/optdiff_bu.png
12evo/drivetrain/kit16/design/optdiff_cl.png
12evo/drivetrain/kit16/design/optdiff_cp.png
12evo/drivetrain/kit16/design/optdiff_cr.png
12evo/drivetrain/kit16/design/optdiff_lf.png
12evo/drivetrain/kit16/design/page1-2.gif
12evo/drivetrain/kit16/fertigung/12evo_ktm_pow_diffhalter_l_h.pdf
12evo/drivetrain/kit16/fertigung/12evo_ktm_pow_diffhalter_r_h.pdf
12evo/drivetrain/kit16/fertigung/12evo_ktm_pow_kettenschutz.pdf
12evo/drivetrain/kit16/fertigung/12evo_ktm_pow_kettenschutz2

aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_024_460mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_025_480mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_026_500mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_027_520mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_028_540mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_029_560mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_030_580mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_031_600mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_032_620mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_033_640mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_034_660mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_035_680mm.png
aerodynamik/unterboden_c/kit17/konzept_c/simulation/vel_y_cut_036_700mm.png
aerodynamik/

elektroauto/batterie/kit15/fertigung/stack/gfk_1_0mm_50x_kit15e_quertraeger_kurz_5_u_7.dxf
elektroauto/batterie/kit15/fertigung/stack/gfk_1_0mm_50x_kit15e_quertraeger_kurz_5_u_7.pdf
elektroauto/batterie/kit15/fertigung/stack/gfk_1_0mm_88x_kit15e_quertraeger_kurz_unten.dxf
elektroauto/batterie/kit15/fertigung/stack/gfk_1_0mm_88x_kit15e_quertraeger_kurz_unten.pdf
elektroauto/batterie/kit15/fertigung/stack/gfk_1_0mm_88x_kit15e_quertraeger_kurz_unten.stp
elektroauto/batterie/kit15/fertigung/stack/gfk_1_5mm_100x_kit15e_seitenbalken.dxf
elektroauto/batterie/kit15/fertigung/stack/gfk_1_5mm_100x_kit15e_seitenbalken.pdf
elektroauto/batterie/kit15/fertigung/stack/gfk_1_5mm_50x_kit15e_quertraeger_1_u_8.dxf
elektroauto/batterie/kit15/fertigung/stack/gfk_1_5mm_50x_kit15e_quertraeger_1_u_8.pdf
elektroauto/batterie/kit15/fertigung/stack/gfk_1_5mm_50x_kit15e_quertraeger_1_u_8.stp
elektroauto/batterie/kit15/fertigung/stack/gfk_1_5mm_88x_kit15e_seitenbalken.dxf
elektroauto/batterie/kit15/fertigung/stack

elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_x.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_x_unten.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_y.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_y_unten.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_z.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_z_detail_ol.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_z_detail_or.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_z_detail_ul.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_z_detail_ur.png
elektroauto/leistungselektronik/kit15/design/analysis/analysis_stress_z_unten.png
elektroauto/leistungselektronik/kit15/design/aradex1.jpg
elektroauto/leistungselektronik/kit15/design/aradex1_ph2.jpg
elektroauto/leistungselektronik/kit15/design/aradex1_vorne.jpg
e

elektroauto/motorengetriebeeinheit/kit18/design_e/spezifische_schmierfilmdicke.jpg
elektroauto/motorengetriebeeinheit/kit18/design_e/stufenplanet/verbindungstool/verbindungstoolschnitt.jpg
elektroauto/motorengetriebeeinheit/kit18/design_e/stufenplanet/zahnwelle/stufenplanet_zahnwelle.jpg
elektroauto/motorengetriebeeinheit/kit18/design_e/stufenplanetauseinemstueck.png
elektroauto/motorengetriebeeinheit/kit18/design_e/stufenplanetkonisch.png
elektroauto/motorengetriebeeinheit/kit18/design_e/stufenplanetmitsteckverzahnung.png
elektroauto/motorengetriebeeinheit/kit18/design_e/stufenplanetverstiftet.png
elektroauto/motorengetriebeeinheit/kit18/design_e/traegheitformel.png
elektroauto/motorengetriebeeinheit/kit18/design_e/v-dichtring.png
elektroauto/motorengetriebeeinheit/kit18/design_e/varilip/reibmomentvarilip.jpg
elektroauto/motorengetriebeeinheit/kit18/design_e/vergleich_mit_und_ohne_kuehlung.png
elektroauto/motorengetriebeeinheit/kit18/design_e/verzahnung/eingriffsstrecke/profil_ueberde

...

monocoque/frontmodul/kit15/fertigungsbilder/img_2060.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2079.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2080.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2081.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2082.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2083.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2084.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2085.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2086.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2087.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2088.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2092.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2093.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2094.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2095.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2096.jpg
monocoque/frontmodul/kit15/fertigungsbilder/img_2099.jpg
monocoque/frontmodul/kit15/fert

monocoque/kit17/formendaten/17_c_ae_sw_form_mw_oben.pdf
monocoque/kit17/formendaten/17_c_form_sk_aussen_l_r.stp
monocoque/kit17/formendaten/17_c_form_sk_aussen_l_r_v2.stp
monocoque/kit17/formendaten/17_c_form_sk_lf_k1_oben_links.stp
monocoque/kit17/formendaten/17_c_form_sk_lf_k1_oben_links_v2.stp
monocoque/kit17/formendaten/17_c_form_sk_lf_k1_oben_rechts.stp
monocoque/kit17/formendaten/17_c_form_sk_lf_k1_oben_rechts_v2.stp
monocoque/kit17/formendaten/17_c_form_sk_lf_k1_unten_links.stp
monocoque/kit17/formendaten/17_c_form_sk_lf_k1_unten_links_v2.stp
monocoque/kit17/formendaten/17_c_form_sk_lf_k1_unten_rechts.stp
monocoque/kit17/formendaten/17_c_form_sk_lf_k1_unten_rechts_v2.stp
monocoque/kit17/formendaten/17c_e_form_sitzunterteil_20170203.stp
monocoque/kit17/formendaten/17c_ilf_links_oben.png
monocoque/kit17/formendaten/17c_ilf_links_unten.png
monocoque/kit17/formendaten/17c_ilf_rechts_oben.png
monocoque/kit17/formendaten/17c_ilf_rechts_unten.png
monocoque/kit17/formendaten/17c_kopfstu

monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_13_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_14_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_15_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_16_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_17_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_18_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_19_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_1_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_20_von_21_.jpg
monocoque/torsionspruefstand/kit17/fotodokumentation/e-tag/torsionspru%cc%88fstand-e_21_von_21_.jpg
m

motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbdinungagb_v4.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbdinungagb_v5.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbindungheckrahmenoben_v2.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbindungheckrahmenoben_v3.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbindungheckrahmenunten_v3.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbindungheckrahmenunten_v4.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbindungheckrahmenunten_v5.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbindungheckrahmenzusaetzlich_v3.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbindungheckrahmenzusaetzlich_v4.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_anbindungheckrahmenzusaetzlich_v5.pdf
motor/kuehlsystem/kit18/teileliste/st37_2mm_2x_18c_mo_kue_blechheckrahmenbackup_v3.pdf
motor/kuehlsystem/kit

wiki/latex/img4ba40072c6990d4c62a0b53c19d973f5.png
wiki/latex/img4bc2128fc053b878f8376c2794935382.png
wiki/latex/img4bced5bb3b69bae043d2dc72ca0ee988.png
wiki/latex/img4bdc8d9bcfb35e1c9bfb51fc69687dfc.png
wiki/latex/img4c085ef88ed1af005968709841e902f1.png
wiki/latex/img4c3c1aa332c7a4165769bddfa688bf7a.png
wiki/latex/img4c5c1de0616c98ba059488ccd76dc80b.png
wiki/latex/img4cd128e3fa971f5c5d60a8108a3b2fd8.png
wiki/latex/img4cfaafd36443675a0f5f3f4fe660cf17.png
wiki/latex/img4d325016ce37049a983901d1aabb7f38.png
wiki/latex/img4d4601e4002b62b902f22fd976aa408e.png
wiki/latex/img4d4e247c01341b8be7b187fcd87814ab.png
wiki/latex/img4dfed4db9b0856af177a42ef2a4e306a.png
wiki/latex/img4e037e0a1872723593230c96ff61a5b3.png
wiki/latex/img4e1c46f4c74e5adf54fbdced4ca50935.png
wiki/latex/img4e5727f4d784156580992da9b0c9f6bb.png
wiki/latex/img4e86faaa910480dc6110899e7112d6c6.png
wiki/latex/img4e9bb1a4c8b1d0be3d46a29efc40e017.png
wiki/latex/img4ec45d4e5263e9516755c76f83947266.png
wiki/latex/img4efd7f5c46999efcb

media which could be deleted. Check one last time if they are linked somewhere

In [101]:
to_delete = [
    "infovault/bilder/2016/fsg",
    "infovault/bilder/2016/fsa",
    "infovault/bilder/2016/fss",
    "infovault/bilder/2016/fsuk",
    "infovault/bilder/2016/rolloutallg",
    "infovault/bilder/2016/sonstiges",
    "infovault/bilder/2016/sponsorentag",
    "infovault/bilder/2016/fsa",
    "infovault/bilder/2016/fsa",    
]
for td in to_delete:
    for n in sorted(allmedia):
        if td in n:
            print(n)

infovault/bilder/2016/fss/tag6/dsc_4323.jpg
infovault/bilder/2016/fsuk/awardceremony/28113490040_bc1c9e7b03_k.jpg
infovault/bilder/2016/fsuk/tag8/dsc_5719.jpg
infovault/bilder/2016/sonstiges/bttf/bttf_shooting.png
infovault/bilder/2016/sonstiges/famielenbild/familienbildpfeilhdr.jpg
infovault/bilder/2016/sonstiges/kit16/breit.jpg
infovault/bilder/2016/sonstiges/rolloutbesuchamz/dsc_8370_2.jpg
infovault/bilder/2016/sonstiges/spassfahrt30.10.2016/dsc_8635.jpg
infovault/bilder/2016/sonstiges/starwarsday/starwars_day.jpg
